In [1]:
import requests
import pandas as pd
import numpy as np
from pulp import LpProblem, LpVariable, LpMaximize, lpSum, LpBinary

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

url = "https://fantasy.premierleague.com/api/bootstrap-static/"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print("Nombre de joueurs récupérés :", len(data["elements"]))
    # Exemple : afficher le nom du premier joueur
    print("Premier joueur :", data["elements"][0]["web_name"])
else:
    print("Erreur lors de la requête :", response.status_code)

Nombre de joueurs récupérés : 740
Premier joueur : Raya


In [2]:
df_teams = pd.DataFrame(data['teams'])[["id", "name", "short_name", "strength", "strength_overall_home", "strength_overall_away", "strength_attack_home", "strength_attack_away", "strength_defence_home", "strength_defence_away"]]

# Ajout d'une colonne AvgStrenght
df_teams["avg_strenght"] = (df_teams["strength_overall_home"] + df_teams["strength_overall_away"]) / 2
df_teams.sort_values(by=["strength", "avg_strenght"] , inplace=True, ascending=False)

In [3]:
df_teams

,id,name,short_name,strength,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,avg_strenght
11,12,Liverpool,LIV,5,1335,1355,1290,1330,1380,1380,1345.0
0,1,Arsenal,ARS,4,1320,1325,1350,1350,1290,1300,1322.5
12,13,Man City,MCI,4,1275,1315,1250,1250,1300,1380,1295.0
6,7,Chelsea,CHE,4,1185,1245,1150,1190,1220,1300,1215.0
14,15,Newcastle,NEW,4,1185,1245,1130,1170,1240,1320,1215.0
1,2,Aston Villa,AVL,3,1125,1250,1110,1200,1140,1300,1187.5
15,16,Nott'm Forest,NFO,3,1165,1205,1150,1230,1180,1180,1185.0
3,4,Bournemouth,BOU,3,1150,1180,1100,1160,1200,1200,1165.0
5,6,Brighton,BHA,3,1150,1175,1090,1140,1210,1210,1162.5
4,5,Brentford,BRE,3,1120,1185,1080,1080,1160,1290,1152.5


In [4]:
df_positions = pd.DataFrame(data['element_types'])[["id", "singular_name_short"]]
df_positions.set_index('id', inplace=True)
df_positions

,singular_name_short
id,
1,GKP
2,DEF
3,MID
4,FWD


In [5]:
# Selection des colonnes pertinentes 
df_players = pd.DataFrame(data['elements'])[['id', 'first_name', 'web_name', 'team', 'element_type', 'now_cost', 'total_points',
'ict_index', 'selected_by_percent', 'selected_rank', 'value_form', 'transfers_in', 'transfers_in_event', 'transfers_out', 'transfers_out_event','minutes', 'expected_goals_per_90', 'saves_per_90',
'expected_assists_per_90', 'expected_goal_involvements_per_90', 'expected_goals_conceded_per_90', 'goals_conceded_per_90',
'defensive_contribution_per_90']]

In [6]:
# Joindre le nom de l'équipe sur la colonne 'team' (qui contient l'ID)
df_players = df_players.merge(df_teams[['id','name', 'short_name']], left_on='team', right_on='id', suffixes=('', '_team'))
# Joindre la position du joueur sur la colonne 'element_type' (ID)
df_players = df_players.merge(df_positions, left_on='element_type', right_on='id', suffixes=('', '_pos'))
# Ajout d'une colonne points total / prix
df_players.insert(loc=6, column="points_per_cost", value=df_players['total_points'] / df_players['now_cost'])
# Tri
df_players.sort_values(by=["points_per_cost", "total_points", "ict_index"] , inplace=True, ascending=False)
# Conversion
df_players['selected_by_percent'] = df_players['selected_by_percent'].astype(str).str.replace(',', '.').astype(float)

# Sélectionner les colonnes intéressantes pour plus de lisibilité
df_players = df_players[[
'id', 'first_name', 'web_name', 'short_name', 'name', 'singular_name_short', 'now_cost', 'total_points', 'points_per_cost',
'ict_index', 'selected_by_percent', 'selected_rank', 'value_form', 'minutes', 'transfers_in', 'transfers_in_event', 'transfers_out', 'transfers_out_event', 'expected_goals_per_90', 
'saves_per_90', 'expected_assists_per_90', 'expected_goal_involvements_per_90', 'expected_goals_conceded_per_90', 'goals_conceded_per_90',
'defensive_contribution_per_90'    
]]

# Renommer pour clarté
df_players = df_players.rename(columns={
    'name': 'team_name',
    'short_name': 'team_short',
    'singular_name_short': 'position',
    'now_cost': 'price'
})

df_players.set_index("id", inplace=True)

In [7]:
df_players[df_players['position'] == 'MID'].head(20)

,first_name,web_name,team_short,team_name,position,price,total_points,points_per_cost,ict_index,selected_by_percent,selected_rank,value_form,minutes,transfers_in,transfers_in_event,transfers_out,transfers_out_event,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,defensive_contribution_per_90
id,,,,,,,,,,,,,,,,,,,,,,,,
241,Moisés,Caicedo,CHE,Chelsea,MID,55,29,0.527273,21.9,7.5,58,1.3,360,355775,69468,197076,6766,0.10,0.0,0.04,0.14,0.96,0.75,14.75
82,Antoine,Semenyo,BOU,Bournemouth,MID,75,36,0.480000,38.3,37.7,3,0.9,360,3626749,283866,368929,8997,0.67,0.0,0.05,0.72,0.87,1.25,7.00
26,Martín,Zubimendi,ARS,Arsenal,MID,54,25,0.462963,15.1,2.4,117,1.1,360,134190,65882,123387,4198,0.05,0.0,0.02,0.07,0.60,0.25,7.25
205,Josh,Cullen,BUR,Burnley,MID,50,23,0.460000,22.4,1.9,126,1.2,332,238359,7440,72736,5273,0.02,0.0,0.14,0.16,2.15,1.63,10.84
237,Enzo,Enzo,CHE,Chelsea,MID,66,29,0.439394,29.7,12.6,35,1.1,348,1106858,65044,209651,24471,0.50,0.0,0.11,0.61,0.98,0.78,6.21
200,Jaidon,Anthony,BUR,Burnley,MID,55,24,0.436364,25.4,1.0,184,1.1,352,101967,9154,30480,2770,0.43,0.0,0.10,0.53,2.10,1.53,8.18
303,James,Garner,EVE,Everton,MID,50,21,0.420000,21.6,0.4,281,1.0,360,38889,4760,13240,1172,0.04,0.0,0.10,0.14,1.54,0.75,11.50
612,Lucas,L.Paquetá,WHU,West Ham,MID,59,23,0.389831,26.0,3.4,91,1.0,337,341561,23573,119668,10735,0.38,0.0,0.06,0.44,1.32,2.94,9.08
593,Pape Matar,P.M.Sarr,TOT,Spurs,MID,50,19,0.380000,17.5,2.7,107,1.0,345,149629,22996,148557,4064,0.08,0.0,0.07,0.15,1.22,0.26,8.35


In [8]:
df_players.to_parquet('df_players.parquet')
df_positions.to_parquet('df_positions.parquet')
df_teams.to_parquet('df_teams.parquet')